# Logistic Regression

In [1]:
import pandas as pd
from IPython.core.display import display
df = pd.read_csv("../data/interim/ecommerce_data-cleaned-0.2.3.csv", index_col=0)
display(df)

,brand,name,description,category_raw,price_raw,discount_raw
0,La Costeï¿½ï¿½a,"La Costena Chipotle Peppers, 7 OZ (Pack of 12)",We aim to show you accurate product informati...,"Food | Meal Solutions, Grains & Pasta | Canned...",31.93,31.93
1,Equate,Equate Triamcinolone Acetonide Nasal Allergy S...,We aim to show you accurate product informati...,Health | Equate | Equate Allergy | Equate Sinu...,10.48,10.48
2,AduroSmart ERIA,AduroSmart ERIA Soft White Smart A19 Light Bul...,We aim to show you accurate product informati...,Electronics | Smart Home | Smart Energy and Li...,10.99,10.99
3,lowrider,"24"" Classic Adjustable Balloon Fender Set Chro...",We aim to show you accurate product informati...,Sports & Outdoors | Bikes | Bike Accessories |...,38.59,38.59
4,Anself,Elephant Shape Silicone Drinkware Portable Sil...,We aim to show you accurate product informati...,Baby | Feeding | Sippy Cups: Alternatives to P...,5.81,5.81
...,...,...,...,...,...,...
29994,NineChef,Sheng Xiang Zhen (ShengXiangZhen) Snack + OneN...,We aim to show you accurate product informati...,"Food | Snacks, Cookies & Chips | Chips & Crisp...",45.99,45.99
29996,Shock Sox,Shock Sox Fork Seal Guards 29-36mm Fork Tube 4...,We aim to show you accurate product informati...,Sports & Outdoors | Bikes | Bike Components | ...,33.25,33.25
29997,Princes,Princes Gooseberries 300g,We aim to show you accurate product informati...,"Food | Meal Solutions, Grains & Pasta | Canned...",8.88,8.88
29998,Create Ion,Create Ion Grace 3/4 Inches Straight Hair Iron...,We aim to show you accurate product informati...,Beauty | Hair Care | Hair Styling Tools | Flat...,50.00,24.50


In [3]:
import numpy as np
def get_range_label(price):
    value = np.round(price, decimals=1)
    if value <= 10:
        return 0
    elif value > 10 and value <= 20:
        return 1
    elif value > 20 and value <= 25:
        return 2
    elif value > 25 and value <= 30:
        return 3
    elif value > 30 and value <= 35:
        return 4
    elif value > 35 and value <= 40:
        return 5
    elif value > 40 and value <= 45:
        return 6
    elif value > 45 and value <= 50:
        return 7
    elif value > 50 and value <= 100:
        return 8
    else:
        return 9


In [4]:
df[labels] = df[price_raw].apply(lambda x: get_range_label(x), axis=1)
display(df)

NameError: name 'price_raw' is not defined

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn import utils

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df.name).toarray()
labels = df.category_id
features.shape

In [18]:
from sklearn.feature_selection import chi2
import numpy as np
N = 2
for name, price_raw in sorted(df.items()):
  features_chi2 = chi2(features, labels == price_raw)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names_out())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("# '{}':".format(name))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

# 'brand':
  . Most correlated unigrams:
. dura
. zoro
  . Most correlated bigrams:
. duro tubes
. dusk dawn
# 'category_raw':
  . Most correlated unigrams:
. dura
. zoro
  . Most correlated bigrams:
. duro tubes
. dusk dawn
# 'description':
  . Most correlated unigrams:
. dura
. zoro
  . Most correlated bigrams:
. duro tubes
. dusk dawn
# 'discount_raw':
  . Most correlated unigrams:
. stansport
. bacati
  . Most correlated bigrams:
. hairspray oz
. metal swing
# 'name':
  . Most correlated unigrams:
. dura
. zoro
  . Most correlated bigrams:
. duro tubes
. dusk dawn
# 'price_raw':
  . Most correlated unigrams:
. dura
. zoro
  . Most correlated bigrams:
. duro tubes
. dusk dawn


In [ ]:
from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
X_train, X_test, y_train, y_test = train_test_split(df['name'], df['price_raw'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=0).fit(X_train_tfidf, y_train)